In [28]:
import pandas as pd
from math import radians
from sklearn.metrics.pairwise import haversine_distances

In [65]:
data = pandas.read_csv('../data/hurr_database/hur.csv', header=0, delimiter=',' ,parse_dates=['date_time'])
data = data.drop('Unnamed: 0', axis=1)
location = data[["Name", "Latitude", "Longitude", "Maximum_sustained_wind_in_knots", "Year"]]
location = location[35683:]

In [66]:
def get_hurricane_data(radius, name, year):
    hurricane = location[(location["Name"] == name) & (location["Year"] == year)]
    hurricane.to_json(f"../data/parsed_data/{name}.json", orient="table")
    atl_counties = pd.read_csv('../data/hurr_database/counties.csv', usecols=["county", "state", "lat", "lon"])
    atl_counties["distance_from_hurr"] = [''] * 731
    harmfulness(atl_counties, hurricane, radius, f"../data/parsed_data/{name}_damage.json")

In [68]:
get_hurricane_data(100, "SANDY", 2012)

51.84946040810249
68.33844566030866
78.85189948846015
38.969091393729485
87.85037235320829
44.39070636503807
78.25195513751312
58.98931293345212
61.446191933804904
46.74803520821778
39.34524930413217
45.98542046759481
68.28059793124869
79.70383038308125
88.79999673927577
29.913895510308688
75.07798176500343
95.60017178707682
94.54947461309678
77.79705936231866
60.02397120622641
46.35435795802956
14.034108411328745
55.72503678098054
61.95379580811297
44.96384556525534
44.33570388544337
16.352717689987983
56.384620903804716
49.05707743477319
54.294114683967145
91.11980913620853
77.87101902804156
20.32813325079998
55.20839131691062
25.39235962559364
42.75229853498751
73.45151828205213
75.56567825897014
82.96619831223613
22.759470512248395
89.61357529873153
93.78326932703605
18.99208237527642
30.893144733873378
75.77932477708349
58.60970293451299
48.77115096694233
73.07064006403203
12.427973886613346
47.5238112013611
53.47779493277124
32.20161601316279
63.33475259994602
38.452837257440905


/usr/local/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
<ipython-input-59-ad346ec5bb5a>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  send_to_jacob["min_distance"] = [0] * send_to_jacob.shape[0]
/usr/local/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:692: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable

In [55]:
def compute_distances_from_county(hurr, lat_county, lon_county):
    lat = list(hurr['Latitude'])
    lon = list(hurr['Longitude'])
    lat_in_radians = [radians(float(_)) for _ in lat]
    lon_in_radians = [radians(float(_)) for _ in lon]
    coords = list(zip(lat_in_radians, lon_in_radians))
    county = list((radians(lat_county), radians(lon_county)))
    distance = list()
    for i in range(0, len(lat_in_radians)):
        distance.append((haversine_distances([list(coords[i]), county])[1,0]) * 6371000/1000)
    return distance

In [59]:
def harmfulness(dataset, hurricane, radius, path):
    count = 0
    for index, row in dataset.iterrows():
        dist = compute_distances_from_county(hurricane, row[2], row[3])
        dataset.at[index, "distance_from_hurr"] = dist
    dataset["harmful"] = ["no"] * dataset.shape[0]
    for i in range(0, len(dataset)):
        if min(dataset["distance_from_hurr"].iloc[i]) < radius:
            print(min(dataset["distance_from_hurr"].iloc[i]))
            dataset["harmful"].iloc[i] = "yes"
    send_to_jacob = dataset[dataset["harmful"] == "yes"]
    send_to_jacob["min_distance"] = [0] * send_to_jacob.shape[0]
    for i in range(0, len(send_to_jacob)):
        send_to_jacob["min_distance"].iloc[i] = min(send_to_jacob["distance_from_hurr"].iloc[i])
    temp = send_to_jacob[["lat", "lon", "min_distance"]].drop_duplicates()
    temp.to_json(path, orient="table")
    print(dataset)
    return temp